In [23]:
import cv2
import mediapipe as mp
import time
import pandas as pd
pd.set_option('display.max_rows', None)
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
def get_right_hand_landmark_indexes(hand, frame):
    index_list = ['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP',
                  'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP',
                  'MIDDLE_FINGER_TIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
                  'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP']

    right_hand_landmarks = []
    for part in index_list:
        if hand.right_hand_landmarks:
            right_hand_landmarks.append([frame, f"{frame}-right_hand-{index_list.index(part)}", "right_hand", index_list.index(part), hand.right_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].x, hand.right_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].y, hand.right_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].z])

        else:
            right_hand_landmarks.append([frame, f"{frame}-right_hand-{index_list.index(part)}", "right_hand",index_list.index(part),  0, 0, 0])

    return right_hand_landmarks

In [4]:
def get_left_hand_landmark_indexes(hand, frame):
    index_list = ['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP',
                  'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP',
                  'MIDDLE_FINGER_TIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
                  'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP']

    left_hand_landmarks = []
    for part in index_list:
        if hand.left_hand_landmarks:
            left_hand_landmarks.append([frame, f"{frame}-left_hand-{index_list.index(part)}", "left_hand", index_list.index(part), hand.left_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].x, hand.left_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].y, hand.left_hand_landmarks.landmark[mp_holistic.HandLandmark(index_list.index(part))].z])

        else:
            left_hand_landmarks.append([frame, f"{frame}-left_hand-{index_list.index(part)}", "left_hand",index_list.index(part),  0, 0, 0])

    return left_hand_landmarks

In [5]:
def get_post_landmark_indexes(pose, frame):
    index_list = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']


    pose_landmarks = []
    for part in index_list:
        if pose.pose_landmarks:
            pose_landmarks.append([frame, f"{frame}-pose-{index_list.index(part)}", "pose", index_list.index(part), pose.pose_landmarks.landmark[mp_holistic.PoseLandmark(index_list.index(part))].x, pose.pose_landmarks.landmark[mp_holistic.PoseLandmark(index_list.index(part))].y, pose.pose_landmarks.landmark[mp_holistic.PoseLandmark(index_list.index(part))].z])

        else:
            pose_landmarks.append([frame, f"{frame}-pose-{index_list.index(part)}", "pose",index_list.index(part),  0, 0, 0])

    return pose_landmarks

In [30]:
def get_face_landmark_indexes(face, frame):
    face_landmarks = []
    for i in range(len(face.face_landmarks.landmark)):
        face_landmarks.append([frame, f"{frame}-face-{i}", "face", i, face.face_landmarks.landmark[i].x, face.face_landmarks.landmark[i].y, face.face_landmarks.landmark[i].z])
    return face_landmarks

In [35]:
def Mediapipe_holistic():
    count = 0
    data = []
    cap = cv2.VideoCapture("WIN_20230427_12_27_08_Pro.mp4")
    #cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        while cap.isOpened():

            success, image = cap.read()

            start = time.time()

            if image is not None:
                img_h, img_w, img_c = image.shape

            # Convert the BGR image to RGB.
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
                image.flags.writeable = False

            # Process the image and find hands
                results = hands.process(image)

                image.flags.writeable = True

            # Draw the hand annotations on the image.
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

                end = time.time()
                totalTime = end - start

                fps = 1 / totalTime
                count = count + 1

                data.append(get_right_hand_landmark_indexes(results, count))
                data.append(get_left_hand_landmark_indexes(results,count))
                data.append(get_post_landmark_indexes(results, count))
                data.append(get_face_landmark_indexes(results, count))
            else:
                cap.release()

            #for live video
            # cv2.putText(image, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2)
            #
            # cv2.imshow('MediaPipe Hands', image)
            #
            # if cv2.waitKey(5) & 0xFF == 27:
            #   break
    return data

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238


In [38]:
data = Mediapipe_holistic()
flat_list = [item for sublist in data for item in sublist]
df = pd.DataFrame(flat_list, columns=['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z'])

     frame           row_id        type  landmark_index         x         y   
0        1   1-right_hand-0  right_hand               0  0.000000  0.000000  \
1        1   1-right_hand-1  right_hand               1  0.000000  0.000000   
2        1   1-right_hand-2  right_hand               2  0.000000  0.000000   
3        1   1-right_hand-3  right_hand               3  0.000000  0.000000   
4        1   1-right_hand-4  right_hand               4  0.000000  0.000000   
5        1   1-right_hand-5  right_hand               5  0.000000  0.000000   
6        1   1-right_hand-6  right_hand               6  0.000000  0.000000   
7        1   1-right_hand-7  right_hand               7  0.000000  0.000000   
8        1   1-right_hand-8  right_hand               8  0.000000  0.000000   
9        1   1-right_hand-9  right_hand               9  0.000000  0.000000   
10       1  1-right_hand-10  right_hand              10  0.000000  0.000000   
11       1  1-right_hand-11  right_hand             